In [ ]:
import os
!sudo apt-get update -qq > /dev/null   
# Install java
!sudo apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! pip install -f nlu pyspark==2.4.7 > /dev/null

In [ ]:
import pandas as pd
import numpy as np
import tqdm
import re
import json
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_feather('sampleindex/sample100000.ftr')

In [ ]:
df.columns

In [ ]:
df = df[['text_reviews_count', 'country_code', 'language_code', 'average_rating', 'description', 'link','num_pages', 'url','image_url',
               'book_id', 'ratings_count', 'title', 'title_without_series']]

In [ ]:
df.head()

In [ ]:
import nlu

In [ ]:
pipe = nlu.load('embed_sentence.bert')

In [ ]:
predictions = pipe.predict(df.description, output_level='document', drop_irrelevant_cols=False, verbose=True)

In [ ]:
predictions

In [ ]:
predictions.info()

In [ ]:
embeddings = []
for i in tqdm.tqdm(df.index):
    embeddings.append( predictions[predictions['description'] == df.description[i]])
#     break

In [ ]:
# embeddings = [predictions[predictions['description'] == df.description[i]] for i in df.index]
def clear_memory():
    gc.collect()
    os.system('sudo sysctl -w vm.drop_caches=3')
    
clear_memory()

In [ ]:
import threading

In [ ]:
# mutithreading version
def func_thread(description, res, index):
    embeddingsres = predictions[predictions['description'] == description]
    res[index] = embeddingsres

In [ ]:
%timeit
thread_list = []
embeddings = {}

breakctr = 0
for i in df.index:
    thread = threading.Thread(target=func_thread, args=(df.description[i], embeddings, i))
    thread_list.append(thread)
    
for thread in thread_list:
    thread.start()
for thread in thread_list:
    thread.join()

In [ ]:
len(embeddings)

In [ ]:
del predictions
del thread_list
del thread

In [ ]:
clear_memory()

In [ ]:
# embeddings_list = [embeddings[i]['embed_sentence_bert_embeddings'] for i in df.index]

In [ ]:
df['embeddings'] = embeddings

In [ ]:
df.head()

In [ ]:
len(df.embeddings.iloc[1])

In [ ]:
embeddings = []
for i in tqdm.tqdm(df.index):
    print(df.description[i])
    print( predictions[predictions['description'] == df.description[i]])
    break

In [ ]:
df['lendesc'] = df.embeddings.apply(lambda x : len(x))

In [ ]:
df

In [ ]:
len(df.embeddings[0])